<a href="https://colab.research.google.com/github/roitraining/SparkforDataEngineers/blob/Development/Ch03_SparkSQL_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
sys.path.append('/home/student/ROI/Spark')
from initspark import *
sc, spark, conf = initspark()

### Homework ##

####First Challenge
Create a Python function to determine if a number is odd or even and use that to select only the even numbered shippers from the TSV folder of northwind. Note the TSV file does not have headers so you will need to do something to make the DataFrame have a meaningful structure. I would suggest using SparkSql as much as possible to rename and cast the columns which are ShipperID, CompanyName, and Phone.

####Second Challenge
Take the Order_LineItems.json folder, read it into a DataFrame, and flatten it and then calculate the average price paid for a product.


In [0]:
o = spark.read.csv('/home/student/ROI/Spark/datasets/northwind/CSVHeaders/orders', header = True, inferSchema = True)
od = spark.read.csv('/home/student/ROI/Spark/datasets/northwind/CSVHeaders/orderdetails', header = True, inferSchema = True)

o.createOrReplaceTempView('Orders')
od.createOrReplaceTempView('OrderDetails')
sql = """
select o.OrderID, o.CustomerID, o.OrderDate
           , COLLECT_SET(NAMED_STRUCT("ProductID", od.ProductID, 
                                      "UnitPrice", od.UnitPrice,
                                      "Quantity", od.Quantity,
                                      "Discount", od.discount)) as LineItems
from Orders as o join OrderDetails as od on o.OrderID = od.OrderID
GROUP BY o.OrderID, o.CustomerID, o.OrderDate
ORDER BY o.OrderID"""
od2 = spark.sql(sql)
od2.write.json('Orders_LineItems.json')



In [0]:
shippers = spark.read.csv('/home/student/ROI/Spark/datasets/northwind/TSV/shippers', header=False, sep = '\t')
#shippers.show()
print(shippers)
shippers.createOrReplaceTempView('shippers')
print(shippers.collect())
shippers = spark.sql('select cast(_c0 as int) as ShipperID, _c1 as ShipperName, _c2 as Phone from shippers')
shippers.createOrReplaceTempView('shippers')
print(shippers)

def isEven(x):
    return x % 2 == 0

from pyspark.sql.functions import udf
from pyspark.sql.types import *

spark.udf.register('isEven', isEven, BooleanType())

spark.sql('select * FROM Shippers WHERE isEven(ShipperID)').show()


In [0]:
o = spark.read.json('/home/student/ROI/Spark/Orders_LineItems.json')
o.createOrReplaceTempView('Orders')
sql = '''select OrderId, CustomerID, OrderDate, l.ProductID, l.UnitPrice, l.Quantity, l.Discount
FROM Orders LATERAL VIEW EXPLODE(LineItems) EXPLODED_TABLE AS l
'''
o2 = spark.sql(sql)
o2.show()
o2.createOrReplaceTempView('FlatOrders')
spark.sql('select avg(UnitPrice) from FlatOrders').show()
